In [132]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, confusion_matrix
import numpy as np 

In [123]:
Data = pd.read_excel("../Python Files/Cleaned_Data.xlsx")
display(Data.head)

<bound method NDFrame.head of      Order ID  Amount  Profit  Quantity   Category Sub-Category  PaymentMode  \
0     B-25696     275    -275         4   Clothing        Saree          COD   
1     B-25687     387    -213         5   Clothing        Saree          UPI   
2     B-25643      50     -44         2   Clothing  Hankerchief          UPI   
3     B-25851     135     -54         5   Clothing        Kurti          COD   
4     B-25756     729    -492         5  Furniture    Bookcases          UPI   
...       ...     ...     ...       ...        ...          ...          ...   
1367  B-25697       4      -3         1   Clothing        Skirt  Credit Card   
1368  B-25873       7       1         1   Clothing  Hankerchief          COD   
1369  B-25740       7       0         2   Clothing        Skirt          COD   
1370  B-25700       7      -3         2   Clothing  Hankerchief          COD   
1371  B-25698       7      -2         1   Clothing  Hankerchief          COD   

      Ord

In [124]:
Nulls = Data.isnull().sum()
print("\nNull Values in Data = ", Nulls)

Cols = Data.columns
print("\nColumns in Data = ", Cols)


Null Values in Data =  Order ID        0
Amount          0
Profit          0
Quantity        0
Category        0
Sub-Category    0
PaymentMode     0
Order Date      0
CustomerName    0
State           0
City            0
Margin          0
dtype: int64

Columns in Data =  Index(['Order ID', 'Amount', 'Profit', 'Quantity', 'Category', 'Sub-Category',
       'PaymentMode', 'Order Date', 'CustomerName', 'State', 'City', 'Margin'],
      dtype='object')


In [125]:
DataNum_Cols = Data.select_dtypes(include=['number']).columns.tolist()
DataCategory_Cols = [col for col in Data.select_dtypes(include=['object']).columns.tolist() if col != 'Order Date']

print("\nNumerics Columns = ", DataNum_Cols)
print("\nCategorical Columns = ", DataCategory_Cols)




Numerics Columns =  ['Amount', 'Profit', 'Quantity', 'Margin']

Categorical Columns =  ['Order ID', 'Category', 'Sub-Category', 'PaymentMode', 'CustomerName', 'State', 'City']


In [126]:
if 'Order Date' in Data.columns:
    Data['Order Date'] = pd.to_datetime(Data['Order Date'])

    # Extract time-based features
    Data['OrderYear'] = Data['Order Date'].dt.year
    Data['OrderMonth'] = Data['Order Date'].dt.month
    Data['OrderDay'] = Data['Order Date'].dt.day
    Data['OrderDayOfWeek'] = Data['Order Date'].dt.dayofweek # Monday=0, Sunday=6
    Data['OrderHour'] = Data['Order Date'].dt.hour
    Data['OrderQuarter'] = Data['Order Date'].dt.quarter
    Data['IsWeekend'] = Data['OrderDayOfWeek'].isin([5, 6]).astype(int)

# Create 'TotalPrice' feature
if 'Quantity' in Data.columns and 'UnitPrice' in Data.columns:
    Data['TotalPrice'] = Data['Quantity'] * Data['UnitPrice']

# Example of creating a 'RevenuePerCustomer' (requires grouping, so this is an aggregation example)
# This would typically be done after initial feature engineering or as part of customer-level features
# if 'CustomerID' in df.columns and 'TotalPrice' in df.columns:
#     customer_revenue = df.groupby('CustomerID')['TotalPrice'].sum().reset_index()
#     customer_revenue.rename(columns={'TotalPrice': 'TotalRevenue'}, inplace=True)
#     df = df.merge(customer_revenue, on='CustomerID', how='left')

# Display the first few rows with new features
# df.head()


/tmp/ipykernel_30000/1185515340.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Data['Order Date'] = pd.to_datetime(Data['Order Date'])


In [127]:
for i in DataCategory_Cols:
    display(Data[i].head(2))

0    B-25696
1    B-25687
Name: Order ID, dtype: object

0    Clothing
1    Clothing
Name: Category, dtype: object

0    Saree
1    Saree
Name: Sub-Category, dtype: object

0    COD
1    UPI
Name: PaymentMode, dtype: object

0    Noopur
1    Sanjna
Name: CustomerName, dtype: object

0      Karnataka
1    Maharashtra
Name: State, dtype: object

0    Bangalore
1       Mumbai
Name: City, dtype: object

In [128]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

DataCategory_Cols = [col for col in Data.select_dtypes(include=['object']).columns.tolist() if col != 'Order ID']

for i in DataCategory_Cols:
    # display(Data[i].value_counts())
    Encoder = LabelEncoder()
    Data[i] = Encoder.fit_transform(Data[i])

for i in DataNum_Cols:
    Scaler = MinMaxScaler()
    Data[i] = Scaler.fit_transform(Data[i].values.reshape(-1, 1))

display(Data.head(2))

,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,Order Date,CustomerName,State,City,Margin,OrderYear,OrderMonth,OrderDay,OrderDayOfWeek,OrderHour,OrderQuarter,IsWeekend
0,B-25696,0.175405,0.520715,0.428571,0,10,0,2018-06-21,180,8,2,0.304892,2018,6,21,3,0,2,0
1,B-25687,0.247896,0.571080,0.571429,0,10,4,2018-06-11,236,11,17,0.401055,2018,6,11,0,0,2,0


In [129]:
display(Data)
X = Data.drop(columns=["IsWeekend", "Order ID", "Order Date"])
Y = Data["IsWeekend"]

display(X.head(4))
display(Y.head(4))

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("X_train:", X_train)
print("X_test:", X_test)
print("Y_train:", Y_train)
print("Y_test:", Y_test)


,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,Order Date,CustomerName,State,City,Margin,OrderYear,OrderMonth,OrderDay,OrderDayOfWeek,OrderHour,OrderQuarter,IsWeekend
0,B-25696,0.175405,0.520715,0.428571,0,10,0,2018-06-21,180,8,2,0.304892,2018,6,21,3,0,2,0
1,B-25687,0.247896,0.571080,0.571429,0,10,4,2018-06-11,236,11,17,0.401055,2018,6,11,0,0,2,0
2,B-25643,0.029773,0.708367,0.142857,0,5,4,2018-04-29,128,7,12,0.330557,2018,4,29,6,0,2,1
3,B-25851,0.084790,0.700244,0.571429,0,6,0,2018-11-06,135,12,13,0.433220,2018,11,6,1,0,4,0
4,B-25756,0.469256,0.344435,0.571429,2,1,4,2018-08-20,157,11,17,0.374425,2018,8,20,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,B-25697,0.000000,0.741673,0.000000,0,12,1,2018-06-22,314,7,12,0.358362,2018,6,22,4,0,2,0
1368,B-25873,0.001942,0.744923,0.000000,0,5,0,2018-11-23,87,10,10,0.549326,2018,11,23,4,0,4,0
1369,B-25740,0.001942,0.744110,0.142857,0,12,0,2018-08-03,216,11,17,0.518771,2018,8,3,4,0,3,0
1370,B-25700,0.001942,0.741673,0.142857,0,5,0,2018-06-25,264,11,17,0.427109,2018,6,25,0,0,2,0


,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,CustomerName,State,City,Margin,OrderYear,OrderMonth,OrderDay,OrderDayOfWeek,OrderHour,OrderQuarter
0,0.175405,0.520715,0.428571,0,10,0,180,8,2,0.304892,2018,6,21,3,0,2
1,0.247896,0.571080,0.571429,0,10,4,236,11,17,0.401055,2018,6,11,0,0,2
2,0.029773,0.708367,0.142857,0,5,4,128,7,12,0.330557,2018,4,29,6,0,2
3,0.084790,0.700244,0.571429,0,6,0,135,12,13,0.433220,2018,11,6,1,0,4


0    0
1    0
2    1
3    0
Name: IsWeekend, dtype: int64

X_train:         Amount    Profit  Quantity  Category  Sub-Category  PaymentMode  \
529   0.087379  0.761170  0.285714         1             0            0   
243   0.199353  0.490658  0.857143         2             2            0   
1309  0.009709  0.750609  0.142857         0             5            1   
664   0.071197  0.750609  0.285714         1             0            2   
745   0.090615  0.738424  0.428571         1             3            0   
...        ...       ...       ...       ...           ...          ...   
1095  0.042071  0.689683  0.428571         0             6            4   
1130  0.017476  0.752234  0.000000         0             6            0   
1294  0.596117  0.380991  0.571429         1             3            0   
860   0.033657  0.744110  0.428571         0             5            0   
1126  0.018770  0.734362  0.571429         0             5            0   

      CustomerName  State  City    Margin  OrderYear  OrderMonth  OrderDay  \
529         

In [137]:
import numpy as np 
from sklearn.metrics import mean_squared_error, r2_score

Model = LinearRegression()
Model.fit(X_train, Y_train)

Y_Pred = Model.predict(X_test)

print("Model Score : ", Model.score(X_test, Y_test)* 100)

print("Root Mean Square Error : ", np.sqrt(mean_squared_error(Y_test, Y_Pred))*100) 

print("R2 Score : ", r2_score(Y_test, Y_Pred)*100)

Train, Test = Model.score(X_train, Y_train)*100, Model.score(X_test, Y_test)*100        

print("Train : ", Train)
print("Test : ", Test)


Model Score :  66.06336783627404
Root Mean Square Error :  27.55997663406813
R2 Score :  66.06336783627404
Train :  62.142810526464174
Test :  66.06336783627404


In [142]:
from sklearn.ensemble import RandomForestRegressor

Model = RandomForestRegressor(random_state = 42)
Model.fit(X_train, Y_train)
Y_Pred = Model.predict(X_test)

print("Model Score : ", Model.score(X_test, Y_test)* 100)
print("Root Mean Square Error : ", np.sqrt(mean_squared_error(Y_test, Y_Pred))*100) 
print("R2 Score : ", r2_score(Y_test, Y_Pred)*100)

Train, Test = Model.score(X_train, Y_train)*100, Model.score(X_test, Y_test)*100        

print("Train : ", Train)
print("Test : ", Test)      

Model Score :  100.0
Root Mean Square Error :  0.0
R2 Score :  100.0
Train :  100.0
Test :  100.0
